Messy code for counting tokens from the experiments.

In [1]:
from collections import Counter

from redel.utils import read_jsonl


# from the docs
def count_tokens(fp):
    tokens_used_prompt = Counter()
    tokens_used_output = Counter()
    for event in read_jsonl(fp):
        if event["type"] == "tokens_used":
            tokens_used_prompt[event["id"]] += event["prompt_tokens"]
            tokens_used_output[event["id"]] += event["completion_tokens"]
    return tokens_used_prompt, tokens_used_output

In [6]:
from pathlib import Path

# define base experiments path
EXPERIMENTS = Path("/Users/andrew/Desktop/Code/kanpai/experiments")

In [10]:
import json


def count_system(fp, root_input_cost_1m, root_output_cost_1m, del_input_cost_1m=0, del_output_cost_1m=0):
    if not fp.exists():
        return
    # get all event paths in system
    event_paths = []
    for result in read_jsonl(fp / "results.jsonl"):
        event_paths.append(fp / Path(result["log_dir"]).stem / "events.jsonl")

    n = len(event_paths)
    total_prompt_tokens_root = 0
    total_output_tokens_root = 0
    total_prompt_tokens_del = 0
    total_output_tokens_del = 0
    # for each run,
    for ep in event_paths:
        # find the root node ID
        with open(ep.parent / "state.json") as f:
            state = json.load(f)
            root_id = state["state"][0]["id"]

        # count tokens by root and by del
        prompt_tokens, output_tokens = count_tokens(ep)
        prompt_tokens_root = prompt_tokens[root_id]
        output_tokens_root = output_tokens[root_id]

        total_prompt_tokens_root += prompt_tokens_root
        total_output_tokens_root += output_tokens_root
        total_prompt_tokens_del += prompt_tokens.total() - prompt_tokens_root
        total_output_tokens_del += output_tokens.total() - output_tokens_root

    # m a t h
    # (I should have used a dict but I did this in like 10 minutes don't judge me)
    avg_prompt_tokens_root = total_prompt_tokens_root / n
    avg_output_tokens_root = total_output_tokens_root / n
    avg_prompt_tokens_del = total_prompt_tokens_del / n
    avg_output_tokens_del = total_output_tokens_del / n

    avg_prompt_cost_root = avg_prompt_tokens_root * root_input_cost_1m / 1000000
    avg_output_cost_root = avg_output_tokens_root * root_output_cost_1m / 1000000
    avg_prompt_cost_del = avg_prompt_tokens_del * del_input_cost_1m / 1000000
    avg_output_cost_del = avg_output_tokens_del * del_output_cost_1m / 1000000

    print(f"========== {fp} ==========")
    print(f"Avg prompt tokens root: {avg_prompt_tokens_root} (${avg_prompt_cost_root})")
    print(f"Avg output tokens root: {avg_output_tokens_root} (${avg_output_cost_root})")
    print(f"Avg prompt tokens del: {avg_prompt_tokens_del} (${avg_prompt_cost_del})")
    print(f"Avg output tokens del: {avg_output_tokens_del} (${avg_output_cost_del})")
    print(f"N: {n}")
    print(
        f"Total cost: ${(avg_prompt_cost_root + avg_output_cost_root + avg_prompt_cost_del + avg_output_cost_del) * n}"
    )

In [12]:
count_system(
    EXPERIMENTS / Path("fanoutqa/dev/trial2/small-leaf"),
    root_input_cost_1m=5,
    root_output_cost_1m=15,
    del_input_cost_1m=0.5,
    del_output_cost_1m=1.5,
)
count_system(
    EXPERIMENTS / Path("travelplanner/validation/small-leaf"),
    root_input_cost_1m=5,
    root_output_cost_1m=15,
    del_input_cost_1m=0.5,
    del_output_cost_1m=1.5,
)
count_system(
    EXPERIMENTS / Path("webarena/test/small-leaf"),
    root_input_cost_1m=5,
    root_output_cost_1m=15,
    del_input_cost_1m=0.5,
    del_output_cost_1m=1.5,
)

========== /Users/andrew/Desktop/Code/kanpai/experiments/fanoutqa/dev/trial2/small-leaf ==========
Avg prompt tokens root: 1913.8733766233765 ($0.009569366883116883)
Avg output tokens root: 363.4642857142857 ($0.0054519642857142865)
Avg prompt tokens del: 28171.0 ($0.0140855)
Avg output tokens del: 509.6818181818182 ($0.0007645227272727273)
N: 308
Total cost: $9.200377000000001
========== /Users/andrew/Desktop/Code/kanpai/experiments/travelplanner/validation/small-leaf ==========
Avg prompt tokens root: 7912.888888888889 ($0.039564444444444444)
Avg output tokens root: 1018.7333333333333 ($0.015281)
Avg prompt tokens del: 9192.044444444444 ($0.0045960222222222215)
Avg output tokens del: 1121.0833333333333 ($0.001681625)
N: 180
Total cost: $11.0021565
========== /Users/andrew/Desktop/Code/kanpai/experiments/webarena/test/small-leaf ==========
Avg prompt tokens root: 55456.268 ($0.27728134)
Avg output tokens root: 171.884 ($0.0025782599999999998)
Avg prompt tokens del: 222370.916 ($0.1111

In [13]:
count_system(EXPERIMENTS / Path("fanoutqa/dev/trial2/baseline"), root_input_cost_1m=5, root_output_cost_1m=15)
count_system(EXPERIMENTS / Path("travelplanner/validation/baseline"), root_input_cost_1m=5, root_output_cost_1m=15)
count_system(EXPERIMENTS / Path("webarena/test/baseline"), root_input_cost_1m=5, root_output_cost_1m=15)

========== /Users/andrew/Desktop/Code/kanpai/experiments/fanoutqa/dev/trial2/baseline ==========
Avg prompt tokens root: 65899.60586319218 ($0.3294980293159609)
Avg output tokens root: 324.8371335504886 ($0.004872557003257329)
Avg prompt tokens del: 0.0 ($0.0)
Avg output tokens del: 0.0 ($0.0)
N: 307
Total cost: $102.65177
========== /Users/andrew/Desktop/Code/kanpai/experiments/travelplanner/validation/baseline ==========
Avg prompt tokens root: 42591.40555555555 ($0.21295702777777775)
Avg output tokens root: 1224.5611111111111 ($0.01836841666666667)
Avg prompt tokens del: 0.0 ($0.0)
Avg output tokens del: 0.0 ($0.0)
N: 180
Total cost: $41.63858
========== /Users/andrew/Desktop/Code/kanpai/experiments/webarena/test/baseline ==========
Avg prompt tokens root: 145537.47430830039 ($0.727687371541502)
Avg output tokens root: 168.6798418972332 ($0.002530197628458498)
Avg prompt tokens del: 0.0 ($0.0)
Avg output tokens del: 0.0 ($0.0)
N: 253
Total cost: $184.745045


In [14]:
total_cost_small_leaf = 9.200377 + 11.0021565 + 98.039051
total_cost_baseline = 102.65177 + 41.63858 + 184.745045
print(total_cost_small_leaf, total_cost_baseline)

118.2415845 329.035395


In [16]:
329.035395 / 118.2415845

2.782738377461442